# Модуль 6

#### 1. Разработка REST API
1. Реализация метода для загрузки данных на сервер -> /upload 
2. Реализация метода для предварительной обработки данных на сервере -> /preprocess 
3. Реализация метода для возврата списка доступных на сервере моделей -> /models 
4. Реализация метода для выбора требуемой модели -> /select_model
5. Реализация метода для выполнения предсказаний по загруженным данным -> /predict

Код REST API с комментариями

In [7]:
from flask import Flask, request, jsonify
import pandas as pd
import pickle
import os
import sys
from io import StringIO
from datetime import datetime


app = Flask(__name__)

# Папка с моделями
MODEL_DIR = 'models'
# Папка с данными 
DATA_DIR = 'data'

# Хранение загруженных данных
data = None
selected_model = None

@app.route('/upload', methods=['POST'])
def upload_data():
    
    global data
    file = request.files['file']
   
    try:
        # Чтение CSV из файлового объекта
        data = pd.read_csv(file.stream)  # Используем file.stream
        
        data.to_csv(f"./{DATA_DIR}/not_process_{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.csv")
        return jsonify({
            "message": "Файл успешно загружен",
      
        }), 200
        
    except pd.errors.ParserError:
        return jsonify({"error": "Ошибка парсинга CSV"}), 400
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/preprocess', methods=['POST'])
def preprocess_data():
    global data
    if data is not None:
        data=data.drop([ '6', '12', '13', '14', '24', '28'],axis=1)
        data=data.iloc[: , 1:]
        data=data.astype('int32')
        data.to_csv(f"./{DATA_DIR}/pre_process_{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.csv")
        return jsonify({"message": "Данные успешно предобработаны"}), 200
    return jsonify({"error": "Данные не предобработаны"}), 400

@app.route('/models', methods=['GET'])
def list_models():
    models = [f for f in os.listdir(MODEL_DIR) if f.endswith('.pkl')]
    return jsonify(models), 200

@app.route('/select_model', methods=['POST'])
def select_model():
    global selected_model
    model_name = request.json.get('model_name')
    if model_name in os.listdir(MODEL_DIR):
        selected_model = model_name
        return jsonify({"message": f"Выбрана модель {model_name}"}), 200
    return jsonify({"error": "Модель не найдена"}), 404

@app.route('/predict', methods=['POST'])
def predict():
    global data, selected_model
    if data is not None and selected_model is not None:
        model_path = os.path.join(MODEL_DIR, selected_model)
        with open(model_path, 'rb') as f:
            model = pickle.load(f)
        predictions = model.predict(data)
        return jsonify(predictions.tolist()), 200
    return jsonify({"error": "Данные не загружены или модель не выбрана, попробуйте ещё"}), 400
'''
if __name__ == '__main__':
    app.run(debug=True)
'''
print()

# 2. РАЗРАБОТКА ТЕЛЕГРАМ-БОТА
1. Телеграм-бот имеет функционал для отправки данных на сервер.  "📤 Загрузить данные"
2. Телеграм-бот имеет функционал для отправки команды для запуска предварительной
обработки данных на сервере "🔧 Предобработка"
3. Телеграм-бот имеет функционал для запроса списка доступных на сервере моделей  "📊 Список моделей"
4. Телеграм-бот имеет функционал для выбора используемой модели "🤖 Выбрать модель"
5. Телеграм-бот имеет функционал для генерации и отображения предсказаний  "🔮 Предсказание"

Код телеграм бота с коментариями

In [8]:

import telebot
from telebot import types
import requests

# Конфигурация
API_TOKEN = '7886805220:AAHSmidDJ6MToG28sIvodK7_HjFRZwHt-zE'
SERVER_URL = 'http://localhost:5000'  

bot = telebot.TeleBot(API_TOKEN)


# Главное меню
def main_keyboard():
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = [
        "📤 Загрузить данные",
        "🔧 Предобработка",
        "📊 Список моделей",
        "🤖 Выбрать модель",
        "🔮 Предсказание"
    ]
    keyboard.add(*buttons)
    return keyboard

@bot.message_handler(commands=['start'])
def start(message):
    welcome_text = (
        "Добро пожаловать! Я ваш ассистент для анализа данных.\n"
        "Выберите действие из меню ниже:"
    )
    bot.send_message(message.chat.id, welcome_text, reply_markup=main_keyboard())

@bot.message_handler(func=lambda message: message.text == "📤 Загрузить данные")
def upload_file(message):
    msg = bot.send_message(message.chat.id, "Пожалуйста, отправьте CSV файл", reply_markup=types.ReplyKeyboardRemove())
    bot.register_next_step_handler(msg, handle_document)

def handle_document(message):
    if message.document is None or not message.document.file_name.endswith('.csv'):
        bot.send_message(message.chat.id, "❌ Неверный формат файла. Отправьте CSV.", reply_markup=main_keyboard())
        return
    
    try:
        file_info = bot.get_file(message.document.file_id)
        downloaded_file = bot.download_file(file_info.file_path)
        
        # Отправка файла на сервер
        response = requests.post(f"{SERVER_URL}/upload", files={'file': downloaded_file})
        if response.status_code == 200:
            bot.send_message(message.chat.id, "✅ Файл успешно загружен!", reply_markup=main_keyboard())
        else:
            bot.send_message(message.chat.id, "❌ Ошибка при загрузке файла", reply_markup=main_keyboard())
    
    except Exception as e:
        bot.send_message(message.chat.id, f"❌ Произошла ошибка: {str(e)}", reply_markup=main_keyboard())

@bot.message_handler(func=lambda message: message.text == "🔧 Предобработка")
def preprocess_data(message):
    try:
        response = requests.post(f"{SERVER_URL}/preprocess")
        bot.send_message(message.chat.id, response.json().get("message", "✅ Предобработка завершена"), reply_markup=main_keyboard())
    except Exception as e:
        bot.send_message(message.chat.id, f"❌ Ошибка: {str(e)}", reply_markup=main_keyboard())

@bot.message_handler(func=lambda message: message.text == "📊 Список моделей")
def show_models(message):
    try:
        response = requests.get(f"{SERVER_URL}/models")
        models = response.json()
        bot.send_message(message.chat.id, f"Доступные модели:\n{', '.join(models)}", reply_markup=main_keyboard())
    except Exception as e:
        bot.send_message(message.chat.id, f"❌ Ошибка: {str(e)}", reply_markup=main_keyboard())

@bot.message_handler(func=lambda message: message.text == "🤖 Выбрать модель")
def select_model_step1(message):
    msg = bot.send_message(message.chat.id, "Введите название модели:", reply_markup=types.ReplyKeyboardRemove())
    bot.register_next_step_handler(msg, select_model_step2)

def select_model_step2(message):
    try:
        model_name = message.text
        response = requests.post(f"{SERVER_URL}/select_model", json={"model_name": model_name})
        bot.send_message(message.chat.id, response.json().get("message", "✅ Модель выбрана"), reply_markup=main_keyboard())
    except Exception as e:
        bot.send_message(message.chat.id, f"❌ Ошибка: {str(e)}", reply_markup=main_keyboard())

@bot.message_handler(func=lambda message: message.text == "🔮 Предсказание")
def make_prediction(message):
    try:
        response = requests.post(f"{SERVER_URL}/predict")
        predictions = response.json()
        bot.send_message(message.chat.id, f"Результаты предсказаний:\n{predictions}", reply_markup=main_keyboard())
    except Exception as e:
        bot.send_message(message.chat.id, f"❌ Ошибка: {str(e)}", reply_markup=main_keyboard())
'''
if __name__ == '__main__':
    bot.polling(none_stop=True)
'''
print()